## Extracting half rink

In [ ]:
from PIL import Image

# Load the image
img = Image.open('../figures/nhl_rink.png')

# Cut image in half to show right side and display
width, height = img.size
new_img = img.crop((width/2, 0, width, height))
new_img.save('../figures/nhl_rink_right.png')

## Graph data

In [1]:
import plotly.graph_objects as go
from ipywidgets import widgets
import numpy as np
from ift6758.visualizations import AdvancedVisualization
from IPython.display import display, clear_output


In [2]:
av = AdvancedVisualization("./../ift6758/data/json_clean/{season}/{season}.pkl")

seasons = [2016, 2017, 2018, 2019, 2020]

# Get all unique teams across the seasons
all_teams = {season : [] for season in seasons}
for season in seasons:
    df = av.load_season_data(season)
    all_teams[season] = sorted(df.team.unique())

z_values = {}
for season in seasons:
    z_values[season] = {team : av.get_plot_args(team, season) for team in all_teams[season]}

In [3]:
season = seasons[1]
default_team = 'Florida Panthers'

grid_size = 100
x = np.linspace(-42.5, 42.5, grid_size + 1)
y = np.linspace(0, 100, grid_size + 1)
z_default = z_values[season][default_team]

colorscale = [
    [0, 'rgb(0, 0, 255)'],
    [0.48, 'rgb(255, 255, 255)'],
    [0.52, 'rgb(255, 255, 255)'],
    [1, 'rgb(255, 0, 0)']
]


In [4]:
heatmap = go.Contour(
        x=x,
        y=y,
        z=z_default,
        colorscale=colorscale,
        showscale=True,
        opacity=1,
        contours=dict(
            start=-1.0,
            end=1.0,
            size=0.075,
            showlines=False
        ),
        zmin=-1.0,
        zmax=1.0,
    )

fig = go.FigureWidget(
    layout=go.Layout(
        width=1000,
        height=1000,
        xaxis=dict(
            range=[-42.5, 42.5],
            autorange=False,
            zeroline=False,
            showgrid=False,
            ticks='',
            showticklabels=True
        ),
        yaxis=dict(
            range=[0, 100],
            autorange=False,
            zeroline=False,
            showgrid=False,
            ticks='',
            showticklabels=True
        ),
        showlegend=True,
        hovermode='closest',
    ),
    data=[heatmap],
)

# Add background of rink image
fig.add_layout_image(
    dict(
        source='../figures/nhl_half_rink_vertical.png',
        xref="x",
        yref="y",
        x=-42.5,
        y=100,
        sizex=85,
        sizey=100,
        sizing="stretch",
        opacity=0.5,
        layer="above"
    )
)

# Add title
fig.update_layout(
    title_text=f'Heatmap of shot differential for {default_team} in {season}',
    title_x=0.5,
    title_font_size=20
)


# Team dropdown selection
team_dropdown = widgets.Dropdown(
    options=all_teams[season],
    value=default_team,
    description='Team:',
)
    
# Update heatmap based on team selection and season selection
def update_heatmap(change):   
    team = team_dropdown.value
    print(f'Updating heatmap for {team} in {season}')
    z = z_values[season][team]
    
    # replace heatmap with new heatmap
    # with fig.batch_update(): # For go.FigureWidget BUT IT DOESN'T FREAKING SHOW UP!!!!!!!!!!!!!!!!!!!!!!!!!!
    #     fig.data[0].z = z
    #     fig.update_layout(
    #         title_text=f'Heatmap of shot differential for {team} in {season}',
    #         title_x=0.5,
    #         title_font_size=20
    #     )
    
    clear_output(wait=True)
    fig.data[0].z = z
    fig.update_layout(
        title_text=f'Heatmap of shot differential for {team} in {season}',
        title_x=0.5,
        title_font_size=20
    )
    display(widgets.HBox([team_dropdown]))
    fig.show()
    
team_dropdown.observe(update_heatmap, names='value')

display(widgets.HBox([team_dropdown, fig]))
fig.show()

## Exporting to HTML with Dash


In [11]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go

heatmap = go.Contour(
        x=x,
        y=y,
        z=z_default,
        colorscale=colorscale,
        showscale=True,
        opacity=1,
        contours=dict(
            start=-1.0,
            end=1.0,
            size=0.075,
            showlines=False
        ),
        zmin=-1.0,
        zmax=1.0,
    )

fig = go.Figure(
    data=[heatmap],
    layout=go.Layout(
        width=1000,
        height=1000,
        xaxis=dict(
            range=[-42.5, 42.5],
            autorange=False,
            zeroline=False,
            showgrid=False,
            ticks='',
            showticklabels=True
        ),
        yaxis=dict(
            range=[0, 100],
            autorange=False,
            zeroline=False,
            showgrid=False,
            ticks='',
            showticklabels=True
        ),
        showlegend=True,
        hovermode='closest',
        images=[dict(
            source='../figures/nhl_half_rink_vertical.png', #this will not work, need to upload it to blog iamge folder
            xref="x",
            yref="y",
            x=-42.5,
            y=100,
            sizex=85,
            sizey=100,
            sizing="stretch",
            opacity=0.5,
            layer="above"
        )]
    )
)

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1(f'Interactive Heatmap of Shot Differential for season {season}'),
    dcc.Dropdown(
        id='team-dropdown',
        options=[{'label': team, 'value': team} for team in all_teams[season]],
        value=default_team,
        clearable=False
    ),
    dcc.Graph(id='heatmap-graph', figure=fig)
])


@app.callback(
    Output('heatmap-graph', 'figure'),
    [Input('team-dropdown', 'value')]
)
def update_figure(selected_team):
    z_new = z_values[season][selected_team]
    fig.data[0].z = z_new
    fig.update_layout(title_text=f'Heatmap of shot differential for {selected_team}')
    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

ValueError: 
    Invalid value of type 'IPython.lib.display.FileLink' received for the 'source' property of layout.image
        Received value: /mnt/Shared Data/Classes/IFT6758/IFT6758-A5-NHL/figures/nhl_half_rink_vertical.png

    The 'source' property is an image URI that may be specified as:
      - A remote image URI string
        (e.g. 'http://www.somewhere.com/image.png')
      - A data URI image string
        (e.g. 'data:image/png;base64,iVBORw0KGgoAAAANSU')
      - A PIL.Image.Image object which will be immediately converted
        to a data URI image string
        See http://pillow.readthedocs.io/en/latest/reference/Image.html
        